<h1 align="center">Prompt Engineering (Engenharia de Prompt)</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Este script está baseado nos tutoriais de [James Briggs](https://www.youtube.com/@jamesbriggs/playlists).

Aqui, exploraremos os fundamentos da `Engenharia de Prompt`. Começaremos instalando a biblioteca `openai`, que usaremos ao longo desses exemplos. No entanto, observe que podemos usar outros `LLMs` aqui, como os oferecidos pelo Cohere ou alternativas de código aberto disponíveis via [Hugging Face](https://huggingface.co/).

In [1]:
%pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Estrutura de um Prompt

Um prompt pode consistir em vários componentes:

* Instruções

* Informação externa ou contexto

* Entrada ou consulta (`query`) do usuário

* Indicador de saída

Nem todos os prompts requerem todos esses componentes, mas geralmente um bom prompt usará dois ou mais deles. Vamos definir o que todos eles são com mais precisão.

As <font color="red">Instruções</font> dizem ao modelo o que fazer, normalmente como ele deve usar entradas e/ou informações externas para produzir a saída que queremos.

<font color="red">Informações externas ou contexto</font> são informações adicionais que inserimos manualmente no prompt, recuperamos por meio de um banco de dados vetorial (memória de longo prazo) ou extraímos por outros meios (chamadas de `API`, cálculos etc.).

A <font color="red">Entrada ou consulta do usuário</font> é normalmente uma consulta inserida diretamente pelo usuário do sistema.

O <font color="red">Indicador de saída</font> é o início do texto gerado. Para um modelo que gera código Python, podemos colocar `import` (já que a maioria dos scripts Python começa com uma biblioteca `import`) ou um `chatbot` pode começar com `Chatbot`:(supondo que formatemos o script do chatbot como linhas de texto intercambiável entre `User` e `Chatbot`).



Cada um desses componentes geralmente deve ser colocado na ordem em que os descrevemos. <font color="orange">Começamos com instruções, fornecemos contexto (se necessário), depois adicionamos a entrada do usuário e, finalmente, terminamos com o indicador de saída.</font>

In [3]:
prompt = """Responda a pergunta com base no contexto abaixo. Se a pergunta não pode ser respondida
usando as informações fornecidas resposta com "não sei".

Context: Large Language Models (LLMs) são os modelos mais recentes usados em NLP.
Seu desempenho superior em relação aos modelos menores os tornou incrivelmente
útil para desenvolvedores que constroem aplicativos habilitados para NLP. Esses modelos
pode ser acessado através da biblioteca 'transformers' do Hugging Face, via OpenAI
usando a biblioteca 'openai', e via Cohere usando a biblioteca 'cohere'.

Question: Quais bibliotecas e provedores de modelos oferecem LLMs?

Answer: """

Neste exemplo, temos:


```
Instruções

Contexto

Pergunta (Input do Usuário)

Indicador de Saída ("Answer: ")
```

Vamos tentar enviar isso para um modelo `GPT-3`. Para isso, você precisará de uma [chave de API OpenAI](https://platform.openai.com/account/api-keys).


Inicializamos um modelo `text-davinci-003`, assim:

In [15]:
import openai


# Obtenha a chave de API no site da OpenAI
#openai.api_key = "OPENAI_API_KEY"


# Isto é quando usas o arquivo .env: 
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY = os.environ['OPENAI_KEY']  
openai.api_key = Eddy_API_KEY 

Carregando a minha chave Key:  True


E faça uma geração a partir do nosso prompt.

In [16]:
# Faço uma query com text-davinci-003
res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256
)


print(res['choices'][0]['text'].strip())

Biblioteca 'transformers' do Hugging Face, biblioteca 'openai' da OpenAI e biblioteca 'cohere' da Cohere.


Como alternativa, se tivermos as informações corretas dentro do `context`, o modelo deve responder com `"Não sei"`, vamos tentar.

In [17]:
prompt = """Responda a pergunta com base no contexto abaixo. Se a pergunta não pode ser respondida
usando as informações fornecida, responda "não sei".

Context: Bibliotecas são lugares cheios de livros.

Question: Quais bibliotecas e provedores de modelos oferecem LLMs?

Answer: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256
)

print(res['choices'][0]['text'].strip())

Não sei.


Perfeito, nossas instruções estão sendo compreendidas pelo modelo. Na maioria dos casos de uso reais, não forneceremos informações externas/contexto para o modelo manualmente. Em vez disso, será um processo automático usando algo como [Long-Term Memory](https://www.pinecone.io/learn/openai-gen-qa/) para recuperar informações relevantes de uma fonte externa.

Por enquanto, isso está além do escopo do que estamos explorando aqui, você pode encontrar mais informações no link acima.

Em resumo, um `prompt` geralmente consiste nesses quatro componentes: instruções, contexto(s), entrada do usuário e o indicador de saída. Agora vamos dar uma olhada na `geração criativa` versus `geração mais rigorosa`.

# Temperatura de Geração (Generation Temperature)

O parâmetro `temperature` usado nos modelos de geração nos diz o quão `"aleatório"` o modelo pode ser. Representa a probabilidade de um modelo escolher uma palavra que não seja a primeira escolha do modelo.

Isso funciona porque o modelo está realmente atribuindo uma previsão de `probabilidade em todos os tokens` dentro de seu vocabulário a cada `"etapa"` (step) do modelo (cada nova palavra ou subpalavra).

TK visual demonstrando etapas sobre tokens

Com cada novo passo adiante, o modelo considera os tokens anteriores alimentados no modelo, cria Embeddings codificando as informações desses tokens em muitas camadas do codificador do modelo e, em seguida, passa essa `codificação` para um `decodificador`. O `decodificador` então prevê a probabilidade de cada token que o modelo conhece (ou seja, está dentro do vocabulário do modelo) com base nas informações codificadas nos Embeddings.

TK visualiza a codificação em um passo de tempo -> decodificação -> previsão sobre muitos tokens

<font color="orange">A uma temperatura do `0` o `decodificador` sempre selecionará o token previsto top (principal, ponto mais alto, etc). A uma temperatura de `1` o modelo sempre selecionará uma palavra que é prevista considerando sua probabilidade atribuída.</font>

TK visualiza a seleção da palavra sempre superior em vários intervalos de tempo quando `temp == 0`, em comparação com a seleção de palavras em vários intervalos de tempo quando `temp == 1`

Considerando tudo isso, se tivermos uma sessão de perguntas e respostas conservadora e baseada em fatos, como no exemplo anterior, faz sentido definir um temperature. <font color="pink">No entanto, se quisermos produzir alguma `escrita criativa` ou `conversas de chatbot`, podemos experimentar e aumentar o número de domínios temperature. 


Vamos tentar!


In [18]:
prompt = """Abaixo está uma conversa com um Chatbot engraçado. As respostas do Chatbot são divertidas e 
engraçadas.

Chatbot: Olá! Eu sou um Chatbot
User: Oi, o que você está fazendo hoje?
Chatbot: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=0.0  # Você pode definir a Temperatura! O default é 1.
)

print(res['choices'][0]['text'].strip())

Estou tentando descobrir como ser mais engraçado. Você tem alguma sugestão?


In [19]:
prompt = """Abaixo está uma conversa com um Chatbot engraçado. As respostas do Chatbot são divertidas e 
engraçadas.

Chatbot: Olá! Eu sou um Chatbot
User: Oi, o que você está fazendo hoje?
Chatbot: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=512,
    temperature=1.0  # Você pode definir a Temperatura! O default é 1.
)

print(res['choices'][0]['text'].strip())

Estou tentando entender como funciona a programação. É como um quebra-cabeça gigante e adorável!


<font color="orange">A segunda resposta é muito mais `criativa` e demonstra o tipo de diferença que podemos esperar entre gerações de baixas e altas `temperature`</font>

# Few-shot Training 

Às vezes, podemos descobrir que um modelo não parece obter o que gostaríamos que fizesse. Podemos ver isso no seguinte exemplo:

In [20]:
prompt = """A seguir, uma conversa com um assistente de IA.
O assistente é tipicamente sarcástico e gracioso, produzindo respostas criativas e engraçadas
às perguntas dos usuários.

User: Qual o significado da vida?
AI: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=1.0
)

print(res['choices'][0]['text'].strip())

Ah, isso é realmente difícil de responder! Uma boa maneira de descobrir é encontrar sua própria definição de significado. Dizem que encontramos significado na vida quando nos dedicamos a viver nossas verdades, amar o que somos e nos engajarmos em algo que estimulamos e é importante para nós.


<font color="orange">Nesse caso, estamos pedindo algo divertido, uma piada em troca de nossa pergunta séria. Mas obtemos uma resposta séria mesmo com o `temperature` para 1.0. Para ajudar o modelo, podemos dar alguns exemplos do tipo de resposta que gostaríamos:</font>

In [21]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. Aqui estão alguns exemplos: 

User: Como você está?
AI: Não posso reclamar, mas às vezes ainda reclamo.

User: Que horas são?
AI: É hora de comprar um relógio.

User: Qual é o sentido da vida?
AI: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=1.0
)

print(res['choices'][0]['text'].strip())

Acho que você precisa encontrar isso por conta própria. Mas se você estiver procurando algo para se orientar, eu diria: "A vida é feita de momentos preciosos - aproveite-os!"


<font color="orange">Esta é uma resposta muito melhor e a maneira como fizemos isso foi fornecendo alguns exemplos que incluíam as entradas e saídas de exemplo que esperávamos. Nós nos referimos a isso como "aprendizagem de poucos tiros" (`few-shot learning`) .</font>

# Adicionando vários contextos

Em alguns casos de uso, como respostas a perguntas (`Question-answering`), podemos usar uma fonte externa de informações para melhorar a confiabilidade ou a veracidade das respostas do modelo. Referimo-nos a essas informações como "conhecimento de origem" (`source knowledge`) , que é qualquer conhecimento inserido no modelo por meio do prompt de entrada.

Criaremos uma lista de informações externas "fictícias" (`dummy`). Na realidade, provavelmente usaríamos memória de longo prazo ([long-term memory](https://www.pinecone.io/learn/openai-gen-qa/)) ou alguma forma de APIs de captura de informações.

In [23]:
contexts = [
    (
        "Os Large Language Models (LLMs) são os modelos mais recentes usados em PNL. " +
        "Seu desempenho superior em relação aos modelos menores os tornou incrivelmente " +
        "útil para desenvolvedores que constroem aplicativos habilitados para NLP. Esses modelos " +
        "pode ser acessado através da biblioteca `transformers` do Hugging Face, via OpenAI " +
        "usando a biblioteca `openai`, e via Cohere usando a biblioteca `cohere`."
    ),
    (
        "Para usar o modelo GPT-3 da OpenAI para tarefas de conclusão (geração), você " +
        "primeiro precisa obter uma chave de API de " +
        "'https://beta.openai.com/account/api-keys'."
    ),
    (
        "A API da OpenAI é acessível via Python usando a biblioteca `openai`. " +
        "Depois de instalar a biblioteca com pip, você pode usá-la da seguinte maneira: \n" +
        "```import openai\nopenai.api_key = 'YOUR_API_KEY'\nprompt = \n" +
        "'<YOUR PROMPT>'\nres = openai.Completion.create(engine='text-davinci" +
        "-003', prompt=prompt, max_tokens=100)\nprint(res)"
    ),
    (
        "A endpoint da OpenAI está disponível para tarefas de completion (conclusão) por meio da " +
        " biblioteca LangChain. Para usá-lo, primeiro instale a biblioteca com " +
        "`pip install langchain openai`. Em seguida, importe a biblioteca e " +
        "inicialize o modelo da seguinte forma: \n" +
        "```from langchain.llms import OpenAI\nopenai = OpenAI(" +
        "model_name='text-davinci-003', openai_api_key='YOUR_API_KEY')\n" +
        "prompt = 'YOUR_PROMPT'\nprint(openai(prompt))```"
    )
]


Nós alimentaríamos esta informação externa em nosso prompt entre as instruções iniciais e a entrada do usuário. Para modelos `OpenAI` é recomendado separar os contextos do resto do prompt usando `###` ou `"""`, e cada contexto independente pode ser separado com algumas novas linhas e `##`, assim:

In [24]:
context_str = '\n\n##\n\n'.join(contexts)

print(f"""Responda à pergunta com base nos contextos abaixo. Se a
pergunta não pode ser respondida usando as informações fornecidas, responda
com "Não sei".

###

Contexts:
{context_str}

###

Question: Dê-me dois exemplos de como usar o modelo GPT-3 da OpenAI
usando Python do início ao fim

Answer: """)

Responda à pergunta com base nos contextos abaixo. Se a
pergunta não pode ser respondida usando as informações fornecidas, responda
com "Não sei".

###

Contexts:
Os Large Language Models (LLMs) são os modelos mais recentes usados em PNL. Seu desempenho superior em relação aos modelos menores os tornou incrivelmente útil para desenvolvedores que constroem aplicativos habilitados para NLP. Esses modelos pode ser acessado através da biblioteca `transformers` do Hugging Face, via OpenAI usando a biblioteca `openai`, e via Cohere usando a biblioteca `cohere`.

##

Para usar o modelo GPT-3 da OpenAI para tarefas de conclusão (geração), você primeiro precisa obter uma chave de API de 'https://beta.openai.com/account/api-keys'.

##

A API da OpenAI é acessível via Python usando a biblioteca `openai`. Depois de instalar a biblioteca com pip, você pode usá-la da seguinte maneira: 
```import openai
openai.api_key = 'YOUR_API_KEY'
prompt = 
'<YOUR PROMPT>'
res = openai.Completion.create(engine='t

In [26]:
prompt = f"""Responda à pergunta com base nos contextos abaixo. Se a
pergunta não pode ser respondida usando as informações fornecidas, responda
com "Não sei".

###

Contexts:
{context_str}

###

Question: Dê-me dois exemplos de como usar o modelo GPT-3 da OpenAI
usando Python do início ao fim

Answer: """



res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=0.0
)

print(res['choices'][0]['text'].strip())

1. Primeiro, obtenha uma chave de API de 'https://beta.openai.com/account/api-keys'. Em seguida, instale a biblioteca `openai` com pip e use-a da seguinte maneira: 
```import openai
openai.api_key = 'YOUR_API_KEY'
prompt = 
'<YOUR PROMPT>'
res = openai.Completion.create(engine='text-davinci-003', prompt=prompt, max_tokens=100)
print(res)```

2. Primeiro, instale a biblioteca `langchain` e `openai` com `pip install langchain openai`. Em seguida, importe a biblioteca e inicialize o modelo da seguinte forma: 
```from langchain.llms import OpenAI
openai = OpenAI(model_name='text-davinci-003', openai_api_key='YOUR_API_KEY')
prompt = 'YOUR_


<font color="orange">Nada mal, mas esses contextos estão realmente ajudando? Talvez o modelo seja capaz de responder a essas perguntas sem a informação adicional (`conhecimento da fonte`), pois é capaz de confiar apenas nas informações armazenadas nos parâmetros internos do modelo (`conhecimento paramétrico`). Vamos perguntar novamente sem a informação externa.</font>

In [27]:
prompt = f"""Responda à pergunta com base nos contextos abaixo. Se a
pergunta não pode ser respondida usando as informações fornecidas, responda
com "Não sei".

Question: Dê-me dois exemplos de como usar o modelo GPT-3 da OpenAI
usando Python do início ao fim

Answer: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=0.0
)

print(res['choices'][0]['text'].strip())

1. Usar o GPT-3 para gerar texto: Primeiro, instale o pacote openai da PyPI usando o comando pip install openai. Em seguida, crie um objeto GPT-3 usando o comando openai.Completion.from_pretrained ('gpt-3'). Por fim, use o método generate para gerar texto a partir do modelo GPT-3.

2. Usar o GPT-3 para classificação de texto: Primeiro, instale o pacote openai da PyPI usando o comando pip install openai. Em seguida, crie um objeto GPT-3 usando o comando openai.Classification.from_pretrained ('gpt-3'). Por fim, use o método classify para classificar o texto usando o modelo GPT-3.


<font color="orange">Estes não são realmente o que pedimos e definitivamente não são muito específicos. Portanto, adicionar algum `conhecimento da fonte` aos nossos prompts pode resultar em resultados muito melhores.</font>